In [ ]:
%matplotlib inline

import numpy as np

import BurstCube

import matplotlib as mpl
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.time import Time

In [ ]:
def loc2channel(x,y):
    '''
    Location to channel conversion.
    
    For a detector type 4 (i.e. calorimeter), Cosima
    return the position of the center of the detector.
    For BurstCube computing the channel is very simple, 
    but this functions allows for generalization
    '''
    
    return 2*(x>0) + (y>0)

In [ ]:
class Hit(object):
    """
    Single interaction with a SiPM, consisting on
    channel number, time of the hit and energy deposited
    """

    def __init__(self, channel, time, energy):

        self.time = time
        self.channel = channel
        self.energy = energy
        
    def __str__(self):
        return "Channel: {}   Time: {:.6f}   Energy: {:.2e}".format(self.channel, self.time, self.energy)
    

In [ ]:
class CosimaSimFile(object):
    '''
    Contains info associated with a Cosima output file (.sim)
    
    Parameters
    ----------
        filename: str
            Path to .sim file. Will be closed on destruction
    
    Attributes
    ----------
        time_start    Start of simulated time
        tot_dur       Total simulated time
        n_thrown      Total number of simulated particles
        n_trigger     Total number of triggered events
        hits          Iterates through all events, returning Hit objects
    
    Warnings
    --------
    Concatenating files with NF and IN keywords is not supported
    '''
    
    def __init__(self, filename):
        
        self.file = open(filename)
    
        in_event_section = False
        
        self.time_start = None
        self.tot_dur = None
        self.n_thrown = None
        
        while True:
        
            line = self.file.readline()
            
            if not line:
                break
            
            if in_event_section:
                if line=='EN\n':
                    in_event_section = False
                
                continue
            
            key = line[0:2] # =\n for empty lines
        
            if key == 'TB':
                self.time_start = float(line.split()[1])*u.second
            elif key == 'SE':
                in_event_section = True
            elif key == 'TE':
                self.tot_dur = float(line.split()[1])*u.second - self.time_start
            elif key == 'TS':
                self.n_thrown = int(line.split()[1])
            elif key == 'NF' or key == 'IN':
                raise ValueError("Not supported") #TODO: fix error type
            
        if self.time_start is None or self.tot_dur is None or self.n_thrown is None:
            raise ValueError("Shouldn't happen") #TODO: fix error type
                
    def __del__(self):
        self.file.close()
        
    def hits(self):
        
        self.file.seek(0)
        
        while True:
        
            line = self.file.readline()

            if not line:
                break
                
            # Time of hit
            if line[0:2] == "TI":
                time = float(line.split()[1]) * u.second

            # Interaction with sensitive detectors
            if line[0:5] != "HTsim":
                continue

            # Format
            x,y,energy = np.array(line.split(';'))[[1,2,4]]

            channel = loc2channel(float(x),float(y))
            energy = float(energy)*u.keV

            # Return
            yield Hit(channel, time, energy)

        

In [ ]:
fSim = CosimaSimFile("sim/bkg.inc1.id1.sim")

print(fSim.time_start)
print(fSim.tot_dur)
print(fSim.n_thrown)

for hit in fSim.hits():
    
    print(hit)